In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
import time
import json
import pandas as pd
import numpy as np
import os
import datetime
from scipy.stats import percentileofscore
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods

In [3]:
df = pd.read_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_logs/IOR/IOR10k_direct_ProM.csv')
df_mini = df[:2000]
df_mini.head()
df_mini.to_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/test_dataset/IOR10k_mini.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_logs/IOR/IOR10k_direct_ProM.csv'

## keep the unique case-event rows with meaningful org:source

In [ ]:
df_diy = pd.read_csv('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_logs/IOR/IOR10k.csv')

In [ ]:
# Assuming the DataFrame is stored in the variable 'df' and the column name is 'concept:instance'
df_diy['concept:instance'] = df_diy['concept:instance'].fillna(0).astype(int)


0              NaN
1              1.0
2              NaN
3              2.0
4              NaN
            ...   
209479    104740.0
209480         NaN
209481    104741.0
209482         NaN
209483    104742.0
Name: concept:instance, Length: 209484, dtype: float64

In [ ]:
df_diy['concept:instance'].unique()

array([        nan, 1.00000e+00, 2.00000e+00, ..., 1.04740e+05,
       1.04741e+05, 1.04742e+05])

In [ ]:
df = df_diy
# Function to check if a value is an integer
def is_integer(x):
    try:
        return int(x) == float(x)
    except:
        return False

# Create a new column that will be used for sorting
df['sort_order'] = df['org:resource'].apply(lambda x: 0 if is_integer(x) else 1 if pd.notna(x) else 2)

# Sort DataFrame by 'sort_order', 'concept:name', 'case:concept:name'
df = df.sort_values(by=['sort_order', 'concept:name', 'case:concept:name'])

# Drop duplicates based on 'concept:name' and 'case:concept:name', keeping the first occurrence
df = df.drop_duplicates(subset=['concept:name', 'case:concept:name'], keep='first')

# Drop the temporary 'sort_order' column
df = df.drop(columns=['sort_order'])

# Resulting DataFrame
df

,concept:name,lifecycle:transition,time:timestamp,case:concept:name,case:LogType,concept:instance,org:resource
33492,Added Activity,assign,2005-03-05 09:43:36.536000+00:00,ior1_0,MXML.EnactmentLog,NaN,Default Resource
40624,Added Activity,assign,2005-03-05 12:00:56.991000+00:00,ior1_1,MXML.EnactmentLog,NaN,Default Resource
20923,Added Activity,assign,2005-03-13 13:06:36.474000+00:00,ior1_100,MXML.EnactmentLog,NaN,Default Resource
37074,Added Activity,assign,2005-03-13 14:35:59.383000+00:00,ior1_101,MXML.EnactmentLog,NaN,Default Resource
24017,Added Activity,assign,2005-03-13 14:11:24.284000+00:00,ior1_102,MXML.EnactmentLog,NaN,Default Resource
...,...,...,...,...,...,...,...
198438,Loan application rejected,assign,2013-05-29 16:20:58.365000+00:00,ior5_988,MXML.EnactmentLog,NaN,NaN
206552,Loan application rejected,assign,2013-03-13 14:24:59.615000+00:00,ior5_99,MXML.EnactmentLog,NaN,NaN
204054,Loan application rejected,assign,2013-06-01 10:04:07.257000+00:00,ior5_991,MXML.EnactmentLog,NaN,NaN
190982,Loan application rejected,assign,2013-06-01 11:11:27.360000+00:00,ior5_993,MXML.EnactmentLog,NaN,NaN


In [ ]:
df['org:resource'].unique()

array(['Default Resource', 'infinite_ressource', nan], dtype=object)

In [ ]:
df.groupby('case').agg({'event': 'count'})

,event
case,
b1_0,12
b1_1,8
b1_10,8
b1_100,12
b1_101,12
...,...
ior5_995,12
ior5_996,12
ior5_997,12


## find drift position

In [4]:
import pandas as pd

def get_drift_locations(file_path):
    # Extracting data name from the file path
    data_name = file_path.split('/')[-1].replace('.csv', '')

    # Reading the CSV file
    df = pd.read_csv(file_path)

    # Finding the unique cases and calculating the drift interval
    num_unique_cases = df['case'].nunique()
    print("Num unique cases:", num_unique_cases)
    drift_interval = num_unique_cases // 10

    # Finding the drift locations based on the drift interval
    drift_locations = []
    case_count = 0
    previous_case = None
    for index, row in df.iterrows():
        current_case = row['case']
        if current_case != previous_case:
            case_count += 1
            if case_count % drift_interval == 0:
                drift_locations.append(index)
        previous_case = current_case

    return data_name, drift_locations

file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_dataset/IRO2500.csv'
data_name, drift_locations = get_drift_locations(file)
print("Data Name:", data_name)
print("Drift Locations:", drift_locations)


Num unique cases: 1500
Data Name: IRO2500
Drift Locations: [1564, 3130, 4878, 6484, 8040, 9728, 11389, 12934, 14575, 16268, 17776, 19378, 20975, 22540, 24132, 25846]


### get monthly batch size

In [5]:
import pandas as pd

def average_monthly_events(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the "completeTime" column to a datetime object
    df['completeTime'] = pd.to_datetime(df['completeTime'], errors='coerce')

    # Extracting the month and year from the "completeTime" column
    df['month_year'] = df['completeTime'].dt.to_period('M')

    # Grouping by the month and year and counting the number of events
    monthly_events = df.groupby('month_year').size()

    # Calculating the average number of events per month
    avg_events_per_month = monthly_events.mean()

    return avg_events_per_month

#file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/synthetic_dataset/ROI5000.csv'
avg_events = average_monthly_events(file)
print("Average Monthly Number of Events:", avg_events)


Average Monthly Number of Events: 24.546511627906977
